# Manejo de Datos Faltantes: Imputacion

### Importar librerias

In [1]:
import janitor  # limpiar datos
import matplotlib.pyplot as plt # visualizacion de datos
import missingno    # exploracion de valores faltantes
import nhanes.load  # libreria con los datos a analizar
import numpy as np  # calculos numericos
import pandas as pd # trabajar con dataframe
import scipy.stats  # trabjar con estadisticos
import seaborn as sns   #   visualizacion estadistica
# import session_info
# sklearn contiene los modelos a utilizar
import sklearn.compose
import sklearn.impute
import sklearn.preprocessing
import statsmodels.api as sm
import statsmodels.datasets
import statsmodels.formula.api as smf

from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.kernel_approximation import Nystroem
from sklearn.linear_model import BayesianRidge, Ridge
from sklearn.neighbors import KNeighborsRegressor
from statsmodels.graphics.mosaicplot import mosaic

### Importar Funciones Personalizadas

In [2]:
# permite utilizar funciones contenidas en otro notebook
%run utils.ipynb

### Configuracion de las Graficas

In [3]:
%matplotlib inline

# tamaño de los graficos
sns.set(
    rc={
        'figure.figsize':(8, 6)
    }
)

# estilo de los graficos
sns.set_style('whitegrid')

## El problema de trabajar con valores faltantes

In [4]:
# trabajamos con un dataset que nos provee statsmodels

airquality_df = (
    sm.datasets.get_rdataset('airquality') # datos de juguete
    .data   # acceder a los datos
    .clean_names(   # limpieza de datos
        case_type = 'snake'
    )
    .add_column('year',1973) # agregamos una columna de indicador
    .assign(    # convertimos una variable a una fecha
        date = lambda df: pd.to_datetime(df[['year', 'month', 'day']])
    )
    .sort_values(by = 'date')   # ordenamos el df segun la fecha
    .set_index('date')  # seteamos el indice con la fecha
)

airquality_df

,ozone,solar_r,wind,temp,month,day,year
date,,,,,,,
1973-05-01,41.0,190.0,7.4,67,5,1,1973
1973-05-02,36.0,118.0,8.0,72,5,2,1973
1973-05-03,12.0,149.0,12.6,74,5,3,1973
1973-05-04,18.0,313.0,11.5,62,5,4,1973
1973-05-05,NaN,NaN,14.3,56,5,5,1973
...,...,...,...,...,...,...,...
1973-09-26,30.0,193.0,6.9,70,9,26,1973
1973-09-27,NaN,145.0,13.2,77,9,27,1973
1973-09-28,14.0,191.0,14.3,75,9,28,1973


In [5]:
# variables que contienen valores faltantes
(
    airquality_df
    .isna()
    .any()
)

ozone       True
solar_r     True
wind       False
temp       False
month      False
day        False
year       False
dtype: bool

Las variables ozone y solar_r contiene valores faltantes

Ajustemos un modelo de regresion lineal con statsmodel sobre los datos

In [6]:
# temperatura en funcion del ozono que contiene valores faltantes
(
    smf.ols(
        formula='temp ~ ozone',
        data=airquality_df
    )
    .fit() # ajuste del modelo sobre los datos
    .summary() # resumen estadistico del modelo
    .tables[0] # muestra solo el contenido de la tabla en la posicion 0
)

Dep. Variable:,temp,R-squared:,0.488
Model:,OLS,Adj. R-squared:,0.483
Method:,Least Squares,F-statistic:,108.5
Date:,"Wed, 12 Apr 2023",Prob (F-statistic):,2.93e-18
Time:,12:39:52,Log-Likelihood:,-386.27
No. Observations:,116,AIC:,776.5
Df Residuals:,114,BIC:,782.1
Df Model:,1,,
Covariance Type:,nonrobust,,


vemos que se aplico el modelo OLS sobre un total de 116 observaciones con un R cuadrado de 0.488. Como se nota tampoco se obtuvo un error al ejecutarse el modelo a pesar que la variable ozono contiene missing. El dataset contiene mas de 150 observaciones por lo que el modelo omitio las observaciones con valores faltantes en la variable ozono

Ahora vamos a considerar otra variable y aplicar el modelo, en este caso solar_r

In [7]:
# temperatura en funcion del ozono y de solar_r
(
    smf.ols(
        formula='temp ~ ozone + solar_r',
        data=airquality_df
    )
    .fit() # ajuste del modelo sobre los datos
    .summary() # resumen estadistico del modelo
    .tables[0] # muestra solo el contenido de la tabla en la posicion 0
)

Dep. Variable:,temp,R-squared:,0.491
Model:,OLS,Adj. R-squared:,0.481
Method:,Least Squares,F-statistic:,52.07
Date:,"Wed, 12 Apr 2023",Prob (F-statistic):,1.47e-16
Time:,12:39:52,Log-Likelihood:,-369.78
No. Observations:,111,AIC:,745.6
Df Residuals:,108,BIC:,753.7
Df Model:,2,,
Covariance Type:,nonrobust,,


En este modelo, tambien se perdieron observaciones ya que se aplico sobre 111. Se obtuvo un valor de R cuadrado de 0.491. Aunque se quisiera comparar ambos modelos mediante su valor de R cuadrado para saber cual de los dos se ajusto mejor, es un error hacerlo debido a que los modelos estan basados sobre diferentes numeros de observaciones.

## Reto: Datos de supervivientes

In [8]:
# dataframe de statsmodels con datos de prueba 
suvirval_df = sm.datasets.get_rdataset('flchain', 'survival').data
suvirval_df

,age,sex,sample.yr,kappa,lambda,flc.grp,creatinine,mgus,futime,death,chapter
0,97,F,1997,5.700,4.860,10,1.7,0,85,1,Circulatory
1,92,F,2000,0.870,0.683,1,0.9,0,1281,1,Neoplasms
2,94,F,1997,4.360,3.850,10,1.4,0,69,1,Circulatory
3,92,F,1996,2.420,2.220,9,1.0,0,115,1,Circulatory
4,93,F,1996,1.320,1.690,6,1.1,0,1039,1,Circulatory
...,...,...,...,...,...,...,...,...,...,...,...
7869,52,F,1995,1.210,1.610,6,1.0,0,4997,0,NaN
7870,52,F,1999,0.858,0.581,1,0.8,0,3652,0,NaN
7871,54,F,2002,1.700,1.720,8,NaN,0,2507,0,NaN
7872,53,F,1995,1.710,2.690,9,NaN,0,4982,0,NaN


In [9]:
# variables con valores faltantes
(
    suvirval_df
    .isna()
    .any()
)

age           False
sex           False
sample.yr     False
kappa         False
lambda        False
flc.grp       False
creatinine     True
mgus          False
futime        False
death         False
chapter        True
dtype: bool

Las variables creatinine y chapter contienen missing

In [10]:
# cantidad de valores faltantes por variable

(
    suvirval_df
    .isnull()
    .sum()
)

age              0
sex              0
sample.yr        0
kappa            0
lambda           0
flc.grp          0
creatinine    1350
mgus             0
futime           0
death            0
chapter       5705
dtype: int64

Hay 1350 missing para la variable creatinine y 5705 para chapter

### Ajusta algunos modelos para predecir si una persona va a sobrevivir o no utilizando a las variables con valores faltantes como predictoras

In [11]:
# Modelo 1
# death en funcion de creatinine

(
    smf.ols(
        formula='death ~ creatinine',
        data=suvirval_df
    )
    .fit() # ajuste del modelo sobre los datos
    .summary() # resumen estadistico del modelo
    .tables[0] # muestra solo el contenido de la tabla en la posicion 0
)

Dep. Variable:,death,R-squared:,0.025
Model:,OLS,Adj. R-squared:,0.025
Method:,Least Squares,F-statistic:,165.0
Date:,"Wed, 12 Apr 2023",Prob (F-statistic):,2.59e-37
Time:,13:15:56,Log-Likelihood:,-4089.4
No. Observations:,6524,AIC:,8183.
Df Residuals:,6522,BIC:,8196.
Df Model:,1,,
Covariance Type:,nonrobust,,


Este modelo solo considera 6524 de las 7874 observaciones que contiene el dataframe con un R cuadrado de 0.025

In [12]:
# Modelo 2
# death en funcion de creatinine y el sexo

(
    smf.ols(
        formula='death ~ creatinine + sex',
        data=suvirval_df
    )
    .fit() # ajuste del modelo sobre los datos
    .summary() # resumen estadistico del modelo
    .tables[0] # muestra solo el contenido de la tabla en la posicion 0
)

Dep. Variable:,death,R-squared:,0.026
Model:,OLS,Adj. R-squared:,0.026
Method:,Least Squares,F-statistic:,88.05
Date:,"Wed, 12 Apr 2023",Prob (F-statistic):,1.85e-38
Time:,13:18:04,Log-Likelihood:,-4084.0
No. Observations:,6524,AIC:,8174.
Df Residuals:,6521,BIC:,8194.
Df Model:,2,,
Covariance Type:,nonrobust,,


El modelo ,2 al igual que el modelo 1, solo considera 6524 observaciones con un R cuadrado de 0.026. Para este caso, ambos modelos pueden compararse debido a que se basan en la misma cantidad de observaciones.